In [13]:
from gibbs.dataclass import GibbsResult
import numpy as np

%load_ext autoreload
%autoreload 2

result = GibbsResult.load("saved_simulations/turbo/_date=20.01.2023_14:51:09")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
display(result.animated_hamiltonian(interval=250,func = np.real))

[0,
 0.3,
 0.6,
 0.8999999999999999,
 1.2,
 1.5,
 1.8,
 2.1,
 2.4,
 2.6999999999999997,
 2.9999999999999996]